In [ ]:
# default_exp background_distributions

# Calculating Background Distributions
In order to differentially assess changes of ions/peptides/proteins between conditions, assumptions about the underlying distributions have to be made. This is necessary to distinguish systematic regulation between conditons from random noise. As introduced in the [MS-EmpiRe paper](https://doi.org/10.1074/mcp.RA119.001509) and refined in the [EmpiReS paper](https://doi.org/10.1101/2020.08.23.234237 ), replicate measurements can be utilized to estimate the empirical *background distributions*, which should reflect the noise underlying the measurements. This approach can then be utilized to:
* estimate the background distribtuions in each condition
* estimate the background in condition1 - condition2 (i.e. the distribution of fold changes between conditions)
* the previous two approaches suffice to perform differential quantification. The approach can however also be used for more complex scenarios. For example, if we want to compare, if two peptides change differently from each other, between two conditions, we can also estimate the underlying background (condition1-condition2)_peptide1 - (condition1-condition2)_peptide2

## Capturing the background of an experimental condition

In the context of AlphaQuant, where proteomics measurements are analysed, we use the word *condition* to describe a set of measurments that was acquired to capture a certain biological state (eg. condition "native" as opposed to conditon "heat shock"). We try to characterize conditions by sorting their ions by mean intensity (intensity is a good predictor for variation, as described in the [MS-EmpiRe paper](https://doi.org/10.1074/mcp.RA119.001509)) and then creating a set of background distributions over the whole intensity range. As a result, we can assign a background distribution to each ion as well as each *differential ion* (i.e. the expected variation in the fold changes of this ion).


In [ ]:
#hide
import alphaquant.diffquant.background_distributions as aqbg

aqbg.ConditionBackgrounds


## Estimate the Background Distribution for a given set of quantified ions
The BackGroundDistribution object is created for a subset of ions. The sets of ions are chosen such that we can assume a certain similarity in how they are distributed (see also notes to the ConditionBackground below). The essential information the BackGroundDistribution object receives is a list of ions with their measured intensity values. The background distribution is constructed from these values in two steps:

1. For each ion, randomly select one of the intensities and subtract if from the other intensites. This rescales the values to fold change values.
2. Subsequently, compare the differences in the rescaled values between different ions. This gives an estimate of the variation of the whole distribution.

We  directly derive some basic properties of the distribution and transform it into a 2d representation (binned fc value vs. counts) for memory efficiency. We also transform the distribution into a cumulative one and subsequently transform this distribution into a z-value distribution for quick lookups

In [ ]:
#hide

aqbg.BackGroundDistribution


In [ ]:
#hide
from numba import jit
from time import time

aqbg.SubtractedBackgrounds


## Utility functions for Background calculation

In [ ]:
#hide
aqbg.get_subtracted_bg



## Unit Tests

In [ ]:
import uuid
import pandas as pd
import numpy as np
def generate_random_input(num_pep,sample2cond_df ):
    pepnames = list(map(lambda _idx : str(uuid.uuid4()), range(num_pep))) #gives uuid strings for each peptide
    randarrays = 10+ 1.5*np.random.randn(num_pep,sample2cond_df.shape[0])
    df_intens = pd.DataFrame(randarrays, columns= sample2cond_df["sample"].tolist())
    df_intens.insert(0, "peptides", pepnames )
    df_intens = df_intens.set_index("peptides")
    return df_intens

sample2cond_df = pd.DataFrame({'sample' : ['A1', 'A2', 'A3', 'B1', 'B2', 'B3'], 'condition' : ['A', 'A', 'A', 'B', 'B', 'B']})
fixed_input = generate_random_input(1000, sample2cond_df)

In [ ]:
#hide


def test_condition_backgrounds(unnormed_df):
    condbg = aqbg.ConditionBackgrounds(unnormed_df, {})
    nonbg_ions = list(set(condbg.ion2nonNanvals.keys()).difference(condbg.ion2background.keys()))
    display(condbg.normed_condition_df)
    
    assert condbg.ion2background.keys() == condbg.ion2nonNanvals.keys()


test_condition_backgrounds(fixed_input)


In [ ]:
#hide
#test subtract background distribution
from scipy.stats import norm
from scipy.stats import normaltest
import matplotlib.pyplot as plt
import numpy as np

idx2nonnanvals = {}

for idx in range(2000):
    nonnanvals =  np.random.normal(loc=0, size=3)
    idx2nonnanvals[idx] = nonnanvals
    
bgdist1 = aqbg.BackGroundDistribution(0, 999, idx2nonnanvals, idx2ion= dict(zip(idx2nonnanvals.keys(), idx2nonnanvals.keys())),p2z={})
bgdist2 = aqbg.BackGroundDistribution(1000, 1999, idx2nonnanvals,idx2ion= dict(zip(idx2nonnanvals.keys(), idx2nonnanvals.keys())), p2z={})

freqdist = aqbg.get_normed_freqs(bgdist1.cumulative)
plt.bar(range(len(freqdist)),freqdist,width=1,color='g')
plt.show()

subtracted_bgs = aqbg.SubtractedBackgrounds(bgdist1, bgdist2, {})

def tranform_fc2count_to_fc_space(fc2counts, num_fcs, rescale_factor):
    fc2counts_fcscales = {}
    for fc, count in fc2counts.items():
        fc2counts_fcscales[fc*rescale_factor] = count/num_fcs

    return fc2counts_fcscales


fc2counts_rescaled = tranform_fc2count_to_fc_space(subtracted_bgs.fc2counts, subtracted_bgs.cumulative[-1],1/100.0)

plt.bar(list(fc2counts_rescaled.keys()), fc2counts_rescaled.values(),width=0.01,color='g')
axes2 = plt.twinx()
x = np.linspace(-4, 4, 1000)
axes2.plot(x, norm.pdf(x, 0, subtracted_bgs.SD)*1.3, color = "red")
axes2.set_ylim(0.0, 0.4)
plt.show()

